### 0. Load data

In [1]:
import os
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

In [2]:
global_mismatch_batch_count = 0

In [12]:
import os
import numpy as np
import torch
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

'''
How data loading works in this project:
* We assume the npy will be sufficient to load into memory
* latent composition:
    concat_tensor = torch.cat([
        x_info['kp'], # 63
        x_info['exp'], # 63,
        x_info['t'], # 3
        x_info['pitch'], # 1
        x_info['yaw'], # 1
        x_info['roll'], # 1
        x_info['scale'], # 1
    ], dim=1)
    
    for person_feat
     - kp single batch avg used for v1. exp keep the same, use global avg scale. t and rot will be subtracted in subsequent frames, 
     - so we can ignore t and rot for now
    for training data
     - use individual kp, exp, subtract dominant t and rot, use global scale
    
    

1. Load both audio and motion latent from the npy files with load_npy_files
    2.  Motion latent is processed to have a window of 65 frames with 10 frames overlap
    2.1 The rotation and translation are frontalized 
3.  The processed data is then loaded into a custom dataset class for random sampling
'''

'''
Part 1: Load and process the npy files
'''
def load_and_process_pair(audio_file, motion_file, latent_type='exp', latent_mask_1=None, latent_bound=None):
    # Load audio file
    # Check the type of audio_file
    if isinstance(audio_file, (str, os.PathLike)):
        # If it's a string (file path), load the audio data
        audio_data = np.load(audio_file)
    elif isinstance(audio_file, (np.ndarray, torch.Tensor)):
        # If it's already a numpy array or torch tensor, use it as is
        audio_data = audio_file
    else:
        # If it's neither a string nor a numpy array/torch tensor, raise an error
        raise ValueError("audio_file must be a file path string, numpy array, or torch tensor")

    # Load and process motion file
    motion_data = np.load(motion_file)
    pad_length = (65 - (motion_data.shape[0] - 10) % 65) % 65
    padded_data = np.pad(motion_data, ((0, pad_length), (0, 0)), mode='constant')
    
    data_without_first_10 = padded_data[10:]
    N = data_without_first_10.shape[0] // 65
    reshaped_data = data_without_first_10[:N*65].reshape(N, 65, 136)
    last_10 = reshaped_data[:, -10:, :]
    prev_10 = np.concatenate([padded_data[:10][None, :, :], last_10[:-1]], axis=0)
    motion_data = np.concatenate([prev_10, reshaped_data], axis=1)
    
    end_indices = torch.ones(N, dtype=torch.int32) * (motion_data.shape[1] - 1)
    end_indices[-1] = motion_data.shape[1] - 1 - pad_length
    
    # Ensure audio and motion data have the same number of frames. 
    # Prev lookup show 1 frame mismatch is common. In this case we only fix batch size mismatch
    min_frames = min(audio_data.shape[0], motion_data.shape[0])
    audio_data = audio_data[:min_frames]
    motion_data = motion_data[:min_frames]
    end_indices = end_indices[:min_frames]
    
    motion_tensor = torch.from_numpy(motion_data)
    if isinstance(audio_data, np.ndarray):
        audio_tensor = torch.from_numpy(audio_data)
    elif isinstance(audio_data, torch.Tensor):
        audio_tensor = audio_data

    motion_tensor, audio_tensor, shape_tensor, mouth_tensor = process_motion_tensor(motion_tensor, audio_tensor, latent_type, latent_mask_1, latent_bound)
    
    return motion_tensor, audio_tensor, shape_tensor, mouth_tensor, end_indices

def process_directory(uid, audio_root, motion_root, latent_type='exp', latent_mask_1=None, latent_bound=None):
    audio_dir = os.path.join(audio_root, uid)
    motion_dir = os.path.join(motion_root, uid)
    
    if not (os.path.isdir(audio_dir) and os.path.isdir(motion_dir)):
        print(f"Directory not found for ", audio_dir, motion_dir)
        return None, None
    
    audio_files = sorted([f for f in os.listdir(audio_dir) if f.endswith('.npy')])
    motion_files = sorted([f for f in os.listdir(motion_dir) if f.endswith('.npy')])
    
    motion_tensor_list = []
    audio_tensor_list = []
    shape_tensor_list = []
    mouth_tensor_list = []
    end_indices_list = []
    for audio_file, motion_file in zip(audio_files, motion_files):
        if audio_file != motion_file and audio_file.split('+')[0] != motion_file.split('+')[0]: 
            print(f"Mismatch in {uid}: {audio_file} and {motion_file}")
            continue
        
        audio_path = os.path.join(audio_dir, audio_file)
        motion_path = os.path.join(motion_dir, motion_file)
        
        motion_tensor, audio_tensor, shape_tensor, mouth_tensor, end_indices = load_and_process_pair(audio_path, motion_path, latent_type, latent_mask_1, latent_bound)
        motion_tensor_list.append(motion_tensor)
        audio_tensor_list.append(audio_tensor)
        shape_tensor_list.append(shape_tensor)
        mouth_tensor_list.append(mouth_tensor)
        end_indices_list.append(end_indices)
    
    return torch.cat(motion_tensor_list, dim=0), torch.cat(audio_tensor_list, dim=0), torch.cat(shape_tensor_list, dim=0), \
            torch.cat(mouth_tensor_list, dim=0), torch.cat(end_indices_list, dim=0)

def load_npy_files(audio_root, motion_root, start_idx=None, end_idx=None, latent_type='exp', latent_mask_1=None, latent_bound=None):
    all_dir_list = sorted(os.listdir(audio_root))
    if start_idx is not None and end_idx is not None:
        dir_list = all_dir_list[start_idx:end_idx]
    else:
        dir_list = all_dir_list
    
    all_motion_data = []
    all_audio_data = []
    all_shape_data = []
    all_mouth_data = []
    all_end_indices = []
                
                
    for uid in tqdm(dir_list, desc="Processing directories"):      
        try:
            motion_data, audio_data, shape_data, mouth_data, end_indices = process_directory(uid, audio_root, motion_root, latent_type, latent_mask_1, latent_bound)
            if audio_data is not None and motion_data is not None:
                    all_motion_data.append(motion_data)
                    all_audio_data.append(audio_data)
                    all_shape_data.append(shape_data)
                    all_mouth_data.append(mouth_data)
                    all_end_indices.append(end_indices)
        except Exception as exc:
            print(f'{uid} generated an exception: {exc}')
    
    motion_tensor = torch.concat(all_motion_data, dim=0)
    audio_tensor = torch.concat(all_audio_data, dim=0)
    shape_tensor = torch.concat(all_shape_data, dim=0)
    mouth_tensor = torch.concat(all_mouth_data, dim=0)
    end_indices = torch.concat(all_end_indices, dim=0)  
    
    # print(f"audio loaded from disk. tensor shape: {audio_tensor.shape}")
    # print(f"motion loaded from disk. tensor shape: {motion_tensor.shape}")
    
    # motion_tensor = normalize_motion_tensor(motion_tensor, latent_bound, latent_mask_1)
    return motion_tensor, audio_tensor, shape_tensor, mouth_tensor, end_indices

def normalize_motion_tensor(motion_tensor, latent_bound=None, latent_mask_1=None):
    motion_tensor = motion_tensor[:, :, :-1] # Remove the last feature which is always 0
    print(f"motion_tensor shape: {motion_tensor.shape}")
    if latent_bound is not None:
        assert len(latent_bound) % 2 == 0
        assert latent_mask_1 is not None
        
        latent_bound = torch.tensor(latent_bound).reshape(-1, 2)
        min_vals = torch.zeros(len(latent_mask_1))
        max_vals = torch.zeros(len(latent_mask_1))
        
        for i, mask_index in enumerate(latent_mask_1):
            min_vals[i] = latent_bound[mask_index][0]
            max_vals[i] = latent_bound[mask_index][1]
    else:
        min_vals, _ = torch.min(motion_tensor.reshape(-1, motion_tensor.shape[-1]), dim=0)
        max_vals, _ = torch.max(motion_tensor.reshape(-1, motion_tensor.shape[-1]), dim=0)

    denominator = max_vals - min_vals
    denominator[denominator == 0] = 1.0  # Set to 1 where max and min are the same

    normalized_tensor = motion_tensor.clone()
    normalized_tensor = 2 * (normalized_tensor - min_vals) / denominator - 1
    normalized_tensor = torch.clamp(normalized_tensor, -1, 1)
    # Calculate quantiles for each feature dimension
    quantiles = torch.tensor([0.0, 0.005, 0.1, 0.25, 0.5, 0.75, 0.9, 0.995, 1.0])
    num_features = normalized_tensor.shape[-1]
    
    print("Quantiles for each feature dimension:")
    for i in range(num_features):
        feat_quantiles = torch.quantile(normalized_tensor[:, i], quantiles)
        print(f"Feature {i}: {feat_quantiles.tolist()}")

    return normalized_tensor

def reverse_normalize_motion_tensorr(motion_tensor, latent_bound, latent_mask_1):
    assert latent_bound is not None and latent_mask_1 is not None
    latent_bound = torch.tensor(latent_bound).reshape(-1, 2)
    full_63_exp = torch.zeros(motion_tensor.shape[0], 68)
    for i, dim in enumerate(latent_mask_1):
        min_bound = latent_bound[dim][0]
        max_bound = latent_bound[dim][1]
        full_63_exp[:, dim] = (motion_tensor[:, i] + 1) * (max_bound - min_bound) / 2 + min_bound
    return full_63_exp.reshape(-1, 68)

'''
Part 2: Frontalize the motion data, swtich euler angles to quaternions
'''

def euler_to_quaternion(pitch, yaw, roll):
    cy = torch.cos(yaw * 0.5)
    sy = torch.sin(yaw * 0.5)
    cp = torch.cos(pitch * 0.5)
    sp = torch.sin(pitch * 0.5)
    cr = torch.cos(roll * 0.5)
    sr = torch.sin(roll * 0.5)

    w = cr * cp * cy + sr * sp * sy
    x = sr * cp * cy - cr * sp * sy
    y = cr * sp * cy + sr * cp * sy
    z = cr * cp * sy - sr * sp * cy

    return torch.stack([w, x, y, z], dim=-1)

def angular_distance_vectorized(poses1, poses2):
    diff = torch.abs(poses1.unsqueeze(1) - poses2.unsqueeze(0))
    diff = torch.min(diff, 2*torch.pi - diff)
    return torch.norm(diff, dim=2)

def find_dominant_pose(poses):
    distances = angular_distance_vectorized(poses, poses)
    total_distances = torch.sum(distances, dim=1)
    min_distance_index = torch.argmin(total_distances)
    return poses[min_distance_index], min_distance_index

def get_rotation_matrix(pitch_, yaw_, roll_):
    """ the input is in degree
    """
    # transform to radian
    pitch = pitch_ / 180 * np.pi
    yaw = yaw_ / 180 * np.pi
    roll = roll_ / 180 * np.pi

    device = pitch.device

    if pitch.ndim == 1:
        pitch = pitch.unsqueeze(1)
    if yaw.ndim == 1:
        yaw = yaw.unsqueeze(1)
    if roll.ndim == 1:
        roll = roll.unsqueeze(1)

    # calculate the euler matrix
    bs = pitch.shape[0]
    ones = torch.ones([bs, 1]).to(device)
    zeros = torch.zeros([bs, 1]).to(device)
    x, y, z = pitch, yaw, roll

    rot_x = torch.cat([
        ones, zeros, zeros,
        zeros, torch.cos(x), -torch.sin(x),
        zeros, torch.sin(x), torch.cos(x)
    ], dim=1).reshape([bs, 3, 3])

    rot_y = torch.cat([
        torch.cos(y), zeros, torch.sin(y),
        zeros, ones, zeros,
        -torch.sin(y), zeros, torch.cos(y)
    ], dim=1).reshape([bs, 3, 3])

    rot_z = torch.cat([
        torch.cos(z), -torch.sin(z), zeros,
        torch.sin(z), torch.cos(z), zeros,
        zeros, zeros, ones
    ], dim=1).reshape([bs, 3, 3])

    rot = rot_z @ rot_y @ rot_x
    return rot.permute(0, 2, 1)  # transpose

@torch.no_grad()
def process_motion_tensor(motion_tensor, audio_tensor, latent_type='exp', latent_mask_1=None, latent_bound=None):
    device = motion_tensor.device
    n_batches, seq_len, _ = motion_tensor.shape
    all_in_bound = torch.ones(n_batches, seq_len, dtype=torch.bool)
    
    return_dominant_headpose = True
    # Extract each component
    kp = motion_tensor[:, :, :63]
    exp = motion_tensor[:, :, 63:126]
    translation = motion_tensor[:, :, 126:129]
    orientation = motion_tensor[:, :, 129:132]
    scale = motion_tensor[:, :, 132:133]
    eye_open_ratio = motion_tensor[:, :, 133:135]
    mouth_open_ratio = motion_tensor[:, :, 135:136]
    
    if return_dominant_headpose:
        '''
        x_d_i_new = scale_tensor * (x_c_s @ R_tensor + exp_tensor) + t_tensor
        
        In version one:
        1. we compute the batch avg of kp used as shape feat
        2. we keep the exp the same
        3. we use the global avg of scale, so constant
        4. we subtract the dominant t and rot from each frame
        '''
        # Find dominant orientation and translation for each frame
        dominant_orientations = torch.zeros((n_batches, 3), device=device)
        dominant_translations = torch.zeros((n_batches, 3), device=device)
        
        for i in range(n_batches):
            dominant_orientations[i], _ = find_dominant_pose(orientation[i])
            dominant_translations[i] = torch.median(translation[i], dim=0).values
        
        # Subtract dominant orientation and translation
        orientation_adjusted = orientation - dominant_orientations.unsqueeze(1)
        translation_adjusted = translation - dominant_translations.unsqueeze(1)
        
    if True:
        '''
        Use exp for motion representation
        '''
        
        motion_tensor = torch.tensor([])
        exp = exp.reshape(n_batches, seq_len, -1)
        if latent_mask_1 is not None:
            for i, d in enumerate(latent_mask_1):
                if d >= 63:
                    continue # skip headpose features
                if i == 0:
                    motion_tensor = exp[:, :, d:d+1]
                else:
                    motion_tensor = torch.cat([motion_tensor, exp[:, :, d:d+1]], dim=2)
            motion_tensor = motion_tensor.reshape(n_batches, seq_len, -1)
        # compute canonical shape kp, using the average of first 5 frames
        first_frame_kp = torch.mean(kp[:, :5, :], dim=1)
        
        # Compute the median of mouth_open_ratio
        median_mouth_open_ratio = torch.median(mouth_open_ratio)
        mouth_open_ratio = median_mouth_open_ratio.expand(n_batches, 1)
        
        if return_dominant_headpose:
            translation_adjusted = translation_adjusted[:, :, :2] # z is always 0
            motion_tensor = torch.cat([motion_tensor, orientation_adjusted, translation_adjusted], dim=2)
        print("motion_tensor.shape", motion_tensor.shape)
        return motion_tensor, audio_tensor, first_frame_kp, mouth_open_ratio


'''
Part 3: Create a custom dataset class for random sampling
'''
class MotionAudioDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        motion_latents, audio_latents, shape_latents, mouth_latents, end_indices = data
        assert len(motion_latents) == len(audio_latents), "Motion and audio latents must have the same length"
        assert len(motion_latents) == len(shape_latents), "Motion and shape latents must have the same length"
        assert len(motion_latents) == len(mouth_latents), "Motion and mouth latents must have the same length"
        assert len(motion_latents) == len(end_indices), "Motion and end indices must have the same length"
        self.motion_latents = motion_latents
        self.audio_latents = audio_latents
        self.shape_latents = shape_latents
        self.mouth_latents = mouth_latents
        self.end_indices = end_indices
        
    def __len__(self):
        return len(self.motion_latents)

    def __getitem__(self, idx):
        return {
            "motion_latent": self.motion_latents[idx], 
            "audio_latent": self.audio_latents[idx],
            "shape_latent": self.shape_latents[idx],
            "mouth_latent": self.mouth_latents[idx],
            "end_indices": self.end_indices[idx]
        }
            
            

In [4]:
audio_root = '/mnt/e/data/live_latent/audio_latent/'
motion_root = '/mnt/e/data/live_latent/motion_temp/'
hdtf_root = '/mnt/e/data/live_latent/hdtf/live_latent/'
hdtf_audio_root = '/mnt/e/data/live_latent/hdtf/audio_latent/'
start_idx = 0
end_idx = 4
hdtf_start_idx = 0
hdtf_end_idx = 337

In [5]:
latent_mask_1 = [ 4, 6, 7, 22, 33, 34, 40, 43, 45, 46, 48, 51, 52, 53, 57, 58, 59, 60, 61, 62 ] # deleted 49,
loss_weight   = [ 1, 1, 1, 1,  2,  3,  1,  1,  2,  3,  2,  1,  1,  1,  1,  2,  1,  1,  3,  1, ]
latent_mask_1 += [i for i in range(62, 62 + 5)] # add 5 headpose features
loss_weight +=  [1, 1, 1, 1, 1] # relative scale is [0.003, 0.002, 0.003, 0.2, 0.2]
latent_bound_list =[
    -0.05029296875,        0.0857086181640625,   -0.07587742805480957,  0.058624267578125,   -0.0004341602325439453,  0.00019466876983642578, 
    -0.038482666015625,    0.0345458984375,      -0.030120849609375,    0.038360595703125,   -3.0279159545898438e-05, 1.3887882232666016e-05,
    -0.0364990234375,      0.036102294921875,    -0.043212890625,       0.046844482421875,   -4.3332576751708984e-05, 1.8775463104248047e-05, 
    -0.03326416015625,     0.057373046875,       -0.03460693359375,     0.031707763671875,   -0.0001958608627319336,  0.0005192756652832031,
    -0.0728759765625,      0.0587158203125,      -0.04840087890625,     0.039642333984375,   -0.00025916099548339844, 0.00048089027404785156, 
    -0.09722900390625,     0.12469482421875,     -0.1556396484375,      0.09326171875,       -0.00018024444580078125, 0.00037860870361328125,
    -0.0279384758323431,   0.010650634765625,    -0.039306640625,       0.03802490234375,    -1.049041748046875e-05,  3.6954879760742188e-06, 
    -0.032989501953125,    0.044281005859375,    -0.037261962890625,    0.0433349609375,     -0.00022792529489379376, 0.0003247261047363281,
    -0.0288234855979681,   0.006015777587890625, -0.0108795166015625,   0.0134124755859375,  -7.784366607666016e-05,  5.2034854888916016e-05, 
    -0.01531982421875,     0.027801513671875,    -0.036041259765625,    0.0242156982421875,  -8.83340835571289e-05,   2.6464462280273438e-05,
    -0.06463623046875,     0.0303802490234375,   -0.0446159653365612,   0.03619384765625,    -0.02947998046875,       0.030792236328125, 
    -0.0159145500510931,   0.018890380859375,    -0.01898193359375,     0.0264739990234375,  -6.103515625e-05,        3.266334533691406e-05,
    -0.0094450069591403,   0.00604248046875,     -0.005710510071367025, 0.00557708740234375, -2.866983413696289e-05,  1.4543533325195312e-05, 
    -0.0265350341796875,   0.01186370849609375,  -0.0227047111839056,   0.01386260986328125, -0.000133514404296875,   6.687641143798828e-05, 
    -0.01129150390625,     0.01331329345703125,  -0.0251922607421875,   0.0195465087890625,  -8.285045623779297e-06,  6.079673767089844e-06, 
    -0.0141599727794528,   0.018341064453125,    -0.0189971923828125,   0.029296875,         -6.049728108337149e-05,  3.057718276977539e-05, 
    -0.01216888427734375,  0.02069091796875,     -0.016754150390625,    0.017974853515625,   -0.00014078617095947266, 6.842613220214844e-05, 
    -0.01910400390625,     0.016204833984375,    -0.025634765625,       0.04150390625,       -0.0100250244140625,     0.00991058349609375, 
    -0.005596160888671875, 0.01132965087890625,  -0.0269775390625,      0.02166748046875,    -0.000362396240234375,   9.059906005859375e-05,
    -0.0325927734375,      0.038818359375,       -0.05877685546875,     0.076416015625,      -0.02215576171875,       0.019775390625, 
    -0.0219573974609375,   0.0247344970703125,   -0.039764404296875,    0.045,               -0.01512908935546875,    0.017730712890625,    
    -21,                   25,                   -30,                   30,                  -23,                     23,
    -0.3,                  0.3,                  -0.06,                 0.28,                
    ]

In [30]:

vox_motion_latents, vox_audio_latents, _, _, end_indices = load_npy_files(audio_root, motion_root, start_idx, end_idx, 'exp', latent_mask_1, latent_bound_list)

Processing directories:   0%|          | 0/4 [00:00<?, ?it/s]

Processing directories:  25%|██▌       | 1/4 [00:00<00:00,  5.38it/s]

motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([7, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])


Processing directories:  50%|█████     | 2/4 [00:00<00:00,  4.93it/s]

motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([6, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([8, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([5, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([

Processing directories: 100%|██████████| 4/4 [00:00<00:00,  6.94it/s]

motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([4, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([2, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])
motion_tensor.shape torch.Size([3, 75, 26])


In [31]:
print(vox_motion_latents.shape)
print("Total number of elements: ", vox_motion_latents.numel())
print("Number of zeros: ", torch.sum(vox_motion_latents == 0))


torch.Size([193, 75, 26])
Total number of elements:  376350
Number of zeros:  tensor(45590)


In [32]:
normalized_motion = normalize_motion_tensor(vox_motion_latents, latent_bound_list, latent_mask_1)

motion_tensor shape: torch.Size([193, 75, 25])
Quantiles for each feature dimension:
Feature 0: [-1.0, -0.8701474666595459, -0.3793734908103943, -0.198322594165802, -0.014149308204650879, 0.1588902473449707, 0.3386431932449341, 0.9498491287231445, 1.0]
Feature 1: [-1.0, -0.8525930047035217, -0.3816734552383423, -0.19672685861587524, -0.01488637924194336, 0.16243517398834229, 0.3457413613796234, 0.9934989213943481, 1.0]
Feature 2: [-1.0, -0.8062981963157654, -0.38438737392425537, -0.20242154598236084, -0.01422119140625, 0.1599721908569336, 0.35126984119415283, 0.9962285757064819, 1.0]
Feature 3: [-1.0, -0.837213933467865, -0.3835735321044922, -0.20214444398880005, -0.011363625526428223, 0.16139960289001465, 0.349974125623703, 0.9685590863227844, 1.0]
Feature 4: [-1.0, -0.8253883123397827, -0.37673240900039673, -0.2001953125, -0.011161446571350098, 0.16015136241912842, 0.34138160943984985, 1.0, 1.0]
Feature 5: [-1.0, -0.8158605098724365, -0.3779662549495697, -0.1990944743156433, -0.00988

In [33]:
normalized_motion = normalized_motion.reshape(-1, 25)

In [34]:
unnormalized_motion = reverse_normalize_motion_tensorr(normalized_motion, latent_bound_list, latent_mask_1)

In [35]:
print(unnormalized_motion.shape)

torch.Size([14475, 68])


In [36]:
unnormalized_motion = unnormalized_motion.reshape(-1, 75, 68)

In [42]:
# compare the unnormalized motion and the original motion
local_unnormalized_motion = unnormalized_motion.reshape(-1, 68)[:, latent_mask_1]
local_vox_motion_latents = vox_motion_latents[:, :, :25].reshape(-1, 25)
print(torch.norm(local_unnormalized_motion - local_vox_motion_latents, dim=1).mean())
# Print the first two elements of vox_motion_latents and unnormalized_motion
print("First elements of vox_motion_latents:")
print(vox_motion_latents[0, 0, :])

print("\nFirst elements of unnormalized_motion:")
print(unnormalized_motion[0, 0, :][latent_mask_1])

# Calculate and print the difference
difference = local_vox_motion_latents[:2] - local_unnormalized_motion[:2]
print("\nDifference between the first two elements:")
print(difference)

# Calculate and print the mean absolute difference
mean_abs_diff = torch.abs(difference).mean()
print(f"\nMean absolute difference: {mean_abs_diff:.6f}")



tensor(0.2863)
First elements of vox_motion_latents:
tensor([-2.0161e-03,  1.2903e-03, -1.3908e-02,  6.9904e-04,  6.1493e-03,
         6.8321e-03, -5.5580e-03, -7.2403e-03, -3.5973e-03, -2.9221e-03,
         3.5095e-03,  4.4441e-03,  5.5199e-03,  1.0109e-03,  5.4207e-03,
        -2.0966e-02, -8.4915e-03,  8.1635e-04,  5.5962e-03,  3.4142e-03,
         3.4142e-03,  5.0877e+00,  1.6544e+00, -6.7100e+00, -6.7963e-02,
         1.3336e-01])

First elements of unnormalized_motion:
tensor([-2.0161e-03,  1.2903e-03, -1.3908e-02,  6.9904e-04,  6.1493e-03,
         6.8321e-03, -5.5580e-03, -7.2403e-03, -3.5973e-03, -2.9221e-03,
         3.5095e-03,  4.4441e-03,  5.5199e-03,  1.0109e-03,  5.4207e-03,
        -2.0966e-02, -8.4915e-03,  8.1635e-04,  5.5962e-03,  3.4142e-03,
         3.4142e-03,  5.0877e+00,  1.6544e+00, -6.7100e+00, -6.7963e-02])

Difference between the first two elements:
tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  

In [ ]:
batch_size, seq_length, _ = vox_motion_latents.shape
mask = torch.arange(seq_length, device=vox_motion_latents.device).expand(batch_size, seq_length) < end_indices.unsqueeze(1)
# Apply the mask to both x_pred and x_gt
vox_motion_latents = vox_motion_latents[mask]
vox_motion_latents = vox_motion_latents.unsqueeze(1)
print(vox_motion_latents.shape)
print("Total number of elements: ", vox_motion_latents.numel())
print("Number of zeros: ", torch.sum(vox_motion_latents == 0))


In [ ]:
min_vals, _ = torch.min(vox_motion_latents.reshape(-1, vox_motion_latents.shape[-1]), dim=(0))
print(min_vals.shape)

In [64]:
min_vals, _ = torch.min(vox_motion_latents.reshape(-1, vox_motion_latents.shape[-1]), dim=(0))
max_vals, _ = torch.max(vox_motion_latents.reshape(-1, vox_motion_latents.shape[-1]), dim=(0))

# Avoid division by zero
denominator = (max_vals - min_vals)
denominator[denominator == 0] = 1.0  # Set to 1 where max and min are the same

vox_motion_latents = 2 * (vox_motion_latents - min_vals) / denominator - 1

# Ensure the tensor is within [-1, 1] range
vox_motion_latents = torch.clamp(vox_motion_latents, -1, 1)

In [65]:
# latent_mask = [i for i in range(57, 63)]
latent_mask = [i for i in range(15, 20)]

# picked_vox_motion_latents = vox_motion_latents[:,:, latent_mask]

In [66]:
# hdtf_audio_latents, hdtf_motion_latents = load_npy_files(hdtf_audio_root, hdtf_root, hdtf_start_idx, hdtf_end_idx)
# hdtf_motion_latents, hdtf_audio_latents, _= process_motion_tensor(hdtf_motion_latents,hdtf_audio_latents, 'exp', latent_mask_1 = latent_mask)

##### Load into motion latents

In [ ]:
# motion_latents = torch.cat([vox_motion_latents, hdtf_motion_latents], dim=0)
motion_latents = torch.cat([vox_motion_latents], dim=0)
print(motion_latents.shape)

### 1. Plot raw latents distribution

1.1 Show quantile

In [ ]:
for i in range(len(latent_mask)):
    quantiles = torch.quantile(motion_latents[:, :, i], torch.tensor([0.0, 0.005, 0.1, 0.25, 0.5, 0.75, 0.9, 0.995, 1.0]))
    std = torch.std(motion_latents[:, :, i])
    print(f"Latent {i}: Quantiles:\n {quantiles.numpy()},\n Std: {std.item()}")
    
    # Find the non-zero quantile
    non_zero_data = motion_latents[:, :, i][motion_latents[:, :, i] != 0]
    quantiles = torch.quantile(non_zero_data, torch.tensor([0.0, 0.005, 0.1, 0.25, 0.5, 0.75, 0.9, 0.995, 1.0]))
    std = torch.std(non_zero_data)
    print(f"Non-zero Latent {i}: Quantiles:\n {quantiles.numpy()},\n Std: {std.item()}")
    print("\n")

print("\n\n\n\n\n\n")

# Print quantiles for vox_orientation_latents
# for i in range(vox_orientation_latents.shape[-1]):
#     quantiles = torch.quantile(vox_orientation_latents[:, i], torch.tensor([0.0, 0.005, 0.1, 0.25, 0.5, 0.75, 0.9, 0.995, 1.0]))
#     std = torch.std(vox_orientation_latents[:, i])
#     print(f"Orientation Latent {i}: Quantiles:\n {quantiles.numpy()},\n Std: {std.item()}")


# # Print quantiles for vox_translation_latents
# for i in range(vox_translation_latents.shape[-1]):
#     quantiles = torch.quantile(vox_translation_latents[:, i], torch.tensor([0.0, 0.005, 0.1, 0.25, 0.5, 0.75, 0.9, 0.995, 1.0]))
#     std = torch.std(vox_translation_latents[:, i])
#     print(f"Translation Latent {i}: Quantiles:\n {quantiles.numpy()},\n Std: {std.item()}")


1.2 Plot in histgram

In [ ]:
# print motion 1
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from tqdm import tqdm
import numpy as np

latents_analysis = motion_latents
# latents_analysis = aligned_latents
latent_dim = len(latent_mask)

# Flatten the batch and length dimensions
flattened_motion = latents_analysis.view(-1, latents_analysis.shape[-1])

# Plot histograms for each feature
column_num = 6
fig, axes = plt.subplots(latent_dim // column_num + int(latent_dim % column_num != 0), column_num, figsize=(20, 14))
axes = axes.flatten()

for i, dim in tqdm(enumerate(latent_mask), total=len(latent_mask), desc="Plotting histograms"):
    print("Plotting feature: ", dim, ", index: ", i)
    data = flattened_motion[:, dim].numpy()
    mean = np.mean(data)
    
    # Plot histogram
    sns.histplot(data, ax=axes[i], kde=True)
    
    # Find the bar that contains the mean and color it red
    for patch in axes[i].patches:
        if patch.get_x() <= mean < patch.get_x() + patch.get_width():
            patch.set_facecolor('red')
            break
    
    print("Plotted feature: ", dim, ", index: ", i)
    axes[i].set_title(f'Feature {dim}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')
    axes[i].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    axes[i].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    
    # Add a vertical line at the mean
    axes[i].axvline(mean, color='red', linestyle='dashed', linewidth=1)

plt.tight_layout()
plt.show()

# # Plot box plots for each feature
# plt.figure(figsize=(20, 6))
# sns.boxplot(data=flattened_motion.numpy())
# plt.title('Box Plot of Motion Features')
# plt.xlabel('Feature Index')
# plt.ylabel('Value')
# plt.gca().xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.show()

1.3 Motion

In [ ]:
# Plot vox_translation and orientation
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from tqdm import tqdm

# Flatten the batch and length dimensions
flattened_vox_translation = vox_translation_latents
flattened_orientation = vox_orientation_latents

# Plot vox_translation
fig, axes = plt.subplots(1, 3, figsize=(20, 5))
axes = axes.flatten()

for i in range(3):
    sns.histplot(flattened_vox_translation[:, i].numpy(), ax=axes[i], kde=True)
    axes[i].set_title(f'Vox Translation {["X", "Y", "Z"][i]}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')
    axes[i].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    axes[i].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))

plt.tight_layout()
plt.show()

# Plot orientation
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
axes = axes.flatten()

for i in range(3):
    sns.histplot(flattened_orientation[:, i].numpy(), ax=axes[i], kde=True)
    axes[i].set_title(f'Orientation {i+1}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')
    axes[i].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    axes[i].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))

plt.tight_layout()
plt.show()

### 2. Clamp at will

In [10]:
target = [0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001]
# 0-6 bound
# -0.05029296875, 0.0857086181640625, -0.07587742805480957, 0.058624267578125, -0.0004341602325439453, 0.00019466876983642578, 
#  -0.038482666015625, 0.0345458984375, -0.030120849609375, 0.038360595703125, -3.0279159545898438e-05, 1.3887882232666016e-05,
# 6-12 bound
# -0.0364990234375, 0.036102294921875, -0.043212890625, 0.046844482421875, -4.3332576751708984e-05, 1.8775463104248047e-05, 
# -0.03326416015625, 0.057373046875, -0.03460693359375, 0.031707763671875, -0.0001958608627319336, 0.0005192756652832031,
# 12-18 bound
# -0.0728759765625, 0.0587158203125, -0.04840087890625, 0.039642333984375, -0.00025916099548339844, 0.00048089027404785156, 
#  -0.09722900390625, 0.12469482421875, -0.1556396484375, 0.09326171875, -0.00018024444580078125, 0.00037860870361328125,
# 18-24 bound
# -0.0279384758323431, 0.010650634765625, -0.039306640625, 0.03802490234375, -1.049041748046875e-05, 3.6954879760742188e-06, 
#  -0.032989501953125, 0.044281005859375, -0.037261962890625, 0.0433349609375, -0.00022792529489379376, 0.0003247261047363281,
# 24-30 bound
# -0.0288234855979681, 0.006015777587890625, -0.0108795166015625, 0.0134124755859375, -7.784366607666016e-05, 5.2034854888916016e-05, 
#  -0.01531982421875, 0.027801513671875, -0.036041259765625, 0.0242156982421875, -8.83340835571289e-05, 2.6464462280273438e-05,
# 30-36 bound
# -0.06463623046875, 0.0303802490234375, -0.0446159653365612, 0.03619384765625, -0.02947998046875, 0.030792236328125, -0.0159145500510931, 
#  0.018890380859375, -0.01898193359375, 0.0264739990234375, -6.103515625e-05, 3.266334533691406e-05
# 36-42 bound
# -0.0094450069591403, 0.00604248046875, -0.005710510071367025, 0.00557708740234375, -2.866983413696289e-05, 1.4543533325195312e-05, 
#  -0.0265350341796875, 0.01186370849609375, -0.0227047111839056, 0.01386260986328125, -0.000133514404296875, 6.687641143798828e-05, 
# 42-48 bound
# -0.01129150390625, 0.01331329345703125, -0.0251922607421875, 0.0195465087890625, -8.285045623779297e-06, 6.079673767089844e-06, 
#  -0.0141599727794528, 0.018341064453125, -0.0189971923828125, 0.029296875, -6.049728108337149e-05, 3.057718276977539e-05, 
# 48-54 bound
# -0.01216888427734375, 0.02069091796875, -0.016754150390625, 0.017974853515625, -0.00014078617095947266, 6.842613220214844e-05, 
#  -0.01910400390625, 0.016204833984375, -0.025634765625, 0.04150390625, -0.0100250244140625, 0.00991058349609375, 
#  -0.005596160888671875, 0.01132965087890625, -0.0269775390625, 0.02166748046875, -0.000362396240234375, 9.059906005859375e-05,


In [ ]:
# Pick a target, generate all_bounds
all_bounds = torch.zeros(len(latent_mask), 2)
for i in range(len(latent_mask)):
    quantiles = torch.quantile(motion_latents[:, :, latent_mask[i]], torch.tensor([target[i], 1-target[i]]))
    all_bounds[i] = quantiles
print(all_bounds)
print([tensor.item() for tensor in all_bounds.reshape(-1)])

In [12]:
# Get all items in bounds if neede to ( deprecated )


# all_in_bound = torch.ones(motion_latents.shape[0], motion_latents.shape[1], dtype=torch.bool)
# for i in range(len(latent_mask)):
#     all_in_bound &= (motion_latents[:, :, i] >= all_bounds[i][0]) & (motion_latents[:, :, i] <= all_bounds[i][1])

# print(all_in_bound.shape)
# print(all_in_bound.sum())
# print(motion_latents.shape[0] * motion_latents.shape[1])

# # Count the number of entries in dim 0 its entries in dim 1 and 2 are all ones
# entries_in_bound_dim0 = all_in_bound.all(dim=1).sum()
# print(entries_in_bound_dim0)

# Count how many of the last dim is full of zeros and drop them
# clamped_motion_latents = clamped_motion_latents.reshape(-1, clamped_motion_latents.shape[-1])
# non_zero_mask = ~(clamped_motion_latents == 0).all(dim=-1)
# clamped_motion_latents = clamped_motion_latents[non_zero_mask]

# # Count the number of dropped vectors
# dropped_count = (~non_zero_mask).sum().item()
# print(f"Number of vectors dropped (all zeros in last dimension): {dropped_count}")

# # Display the original and new shapes for reference
# print(f"Original shape: {clamped_motion_latents.shape[0] + dropped_count, clamped_motion_latents.shape[1]}")
# print(f"New shape after dropping zero vectors: {clamped_motion_latents.shape}")


In [13]:
# Clamp motion tensor to boundary
clamped_motion_latents = torch.zeros_like(motion_latents)

for i in range(len(latent_mask)):
    lower_bound = all_bounds[i][0]
    upper_bound = all_bounds[i][1]
    clamped_motion_latents[:, :, latent_mask[i]] = torch.clamp(motion_latents[:, :, latent_mask[i]], min=lower_bound, max=upper_bound)

In [ ]:
# print audio latents analysis ( deprecated )
'''
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

latents_analysis = audio_latents[:, :, :]
latent_dim = latents_analysis.shape[-1]

# Flatten the batch and length dimensions
flattened_motion = latents_analysis.view(-1, latent_dim)

# Calculate basic statistics
mean = flattened_motion.mean(dim=0)
std = flattened_motion.std(dim=0)
min_vals = flattened_motion.min(dim=0).values
max_vals = flattened_motion.max(dim=0).values

# Calculate absolute magnitude statistics
abs_flattened_motion = flattened_motion.abs()
abs_mean = abs_flattened_motion.mean(dim=0)
abs_std = abs_flattened_motion.std(dim=0)
abs_min_vals = abs_flattened_motion.min(dim=0).values
abs_max_vals = abs_flattened_motion.max(dim=0).values

# Create a summary dataframe
summary = pd.DataFrame({
    'Mean': mean.numpy(),
    'Std': std.numpy(),
    'Min': min_vals.numpy(),
    'Max': max_vals.numpy(),
    'Abs Mean': abs_mean.numpy(),
    'Abs Std': abs_std.numpy(),
    'Abs Min': abs_min_vals.numpy(),
    'Abs Max': abs_max_vals.numpy()
})

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Print the full summary
print(summary.to_string())

# # Plot histograms for each feature
# fig, axes = plt.subplots(latent_dim // 10 + int(latent_dim % 10 != 0), 10, figsize=(20, 14))
# axes = axes.flatten()

# for i in tqdm(range(latent_dim), desc="Plotting histograms"):
#     sns.histplot(flattened_motion[:, i].numpy(), ax=axes[i], kde=True)
#     axes[i].set_title(f'Feature {i}')
#     axes[i].set_xlabel('')
#     axes[i].set_ylabel('')
#     axes[i].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
#     axes[i].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))

# plt.tight_layout()
# plt.show()

# # Plot box plots for each feature
# plt.figure(figsize=(20, 6))
# sns.boxplot(data=flattened_motion.numpy())
# plt.title('Box Plot of Motion Features')
# plt.xlabel('Feature Index')
# plt.ylabel('Value')
# plt.gca().xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.show()
'''

In [ ]:
clamped_motion_latents.shape

In [ ]:
# print motion 1
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
import pandas as pd
from tqdm import tqdm

latents_analysis = clamped_motion_latents
# latents_analysis = aligned_latents
latent_dim = len(latent_mask)

# Flatten the batch and length dimensions
flattened_motion = latents_analysis.view(-1, latents_analysis.shape[-1])
print(flattened_motion.shape)
# # Calculate basic statistics
# mean = flattened_motion.mean(dim=0)
# std = flattened_motion.std(dim=0)
# min_vals = flattened_motion.min(dim=0).values
# max_vals = flattened_motion.max(dim=0).values

# # Calculate absolute magnitude statistics
# abs_flattened_motion = flattened_motion.abs()
# abs_mean = abs_flattened_motion.mean(dim=0)
# abs_std = abs_flattened_motion.std(dim=0)
# abs_min_vals = abs_flattened_motion.min(dim=0).values
# abs_max_vals = abs_flattened_motion.max(dim=0).values

# # Create a summary dataframe
# summary = pd.DataFrame({
#     'Mean': mean.numpy(),
#     'Std': std.numpy(),
#     'Min': min_vals.numpy(),
#     'Max': max_vals.numpy(),
#     'Abs Mean': abs_mean.numpy(),
#     'Abs Std': abs_std.numpy(),
#     'Abs Min': abs_min_vals.numpy(),
#     'Abs Max': abs_max_vals.numpy()
# })

# # Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', None)

# # Print the full summary
# print(summary.to_string())

# Plot histograms for each feature
column_num = 6
fig, axes = plt.subplots(latent_dim // column_num + int(latent_dim % column_num != 0), column_num, figsize=(20, 14))
axes = axes.flatten()

for i, dim in tqdm(enumerate(latent_mask), total=len(latent_mask), desc="Plotting histograms"):
    sns.histplot(flattened_motion[:, dim].numpy(), ax=axes[i], kde=True)
    axes[i].set_title(f'Feature {dim}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')
    axes[i].xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
    axes[i].yaxis.set_major_formatter(ScalarFormatter(useMathText=True))

plt.tight_layout()
plt.show()

# # Plot box plots for each feature
# plt.figure(figsize=(20, 6))
# sns.boxplot(data=flattened_motion.numpy())
# plt.title('Box Plot of Motion Features')
# plt.xlabel('Feature Index')
# plt.ylabel('Value')
# plt.gca().xaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.gca().yaxis.set_major_formatter(ScalarFormatter(useMathText=True))
# plt.show()

### 3. Deprecated Hard scaling. Likely Won't need.


In [ ]:
# Break execution
1 = 1

In [ ]:
import numpy as np
import torch

# Assuming abs_std is already defined
# abs_std = torch.randn(70)  # Example tensor, replace with your actual tensor

# Define the bins in logarithmic scale from 1e-1 to 1e-8
bins = np.logspace(-1, -8, num=8)

# Digitize the abs_std values into the defined bins
bin_indices = np.digitize(abs_std[:63].numpy(), bins)

# Create a list to store indices for each bin
bin_index_lists = [[] for _ in range(len(bins) + 1)]

# Populate the bin_index_lists
for idx, bin_idx in enumerate(bin_indices):
    bin_index_lists[bin_idx].append(idx)

# Print the bin counts and indices
bin_counts = np.bincount(bin_indices, minlength=len(bins) + 1)
for i, (count, indices) in enumerate(zip(bin_counts, bin_index_lists)):
    if i == 0:
        print(f"< {bins[0]:.1e}: {count}")
    elif i == len(bins):
        print(f">= {bins[-1]:.1e}: {count}")
    else:
        print(f"{bins[i-1]:.1e} - {bins[i]:.1e}: {count}")
    print(f"Indices: {indices}")
    print()

In [ ]:
latents_analysis = motion_latents
latent_dim = latents_analysis.shape[-1]

# Flatten the batch and length dimensions
flattened_motion = latents_analysis.view(-1, latent_dim)

flattened_motion.shape

In [ ]:
def normalize_features(features):
    # Step 1: Standardize (mean=0, std=1)
    mean = features.mean(dim=0, keepdim=True)
    std = features.std(dim=0, keepdim=True)
    standardized = (features - mean) / (std + 1e-8)
    
    # Step 2: Min-max scaling to [0, 1]
    # min_vals = standardized.min(dim=0, keepdim=True)[0]
    # max_vals = standardized.max(dim=0, keepdim=True)[0]
    # normalized = (standardized - min_vals) / (max_vals - min_vals + 1e-8)
    
    return standardized

flattened_motion = normalize_features(flattened_motion)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

latents_analysis = motion_latents

# Flatten the batch and length dimensions
flattened_motion = latents_analysis.view(-1, latent_dim)
flattened_motion = flattened_motion[:, :63]
latent_dim = flattened_motion.shape[-1]
# Calculate basic statistics
mean = flattened_motion.mean(dim=0)
std = flattened_motion.std(dim=0)
min_vals = flattened_motion.min(dim=0).values
max_vals = flattened_motion.max(dim=0).values

# Create a summary dataframe
import pandas as pd
summary = pd.DataFrame({
    'Mean': mean.numpy(),
    'Std': std.numpy(),
    'Min': min_vals.numpy(),
    'Max': max_vals.numpy()
})

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Print the full summary
print(summary.to_string())

# Plot histograms for each feature
fig, axes = plt.subplots(latent_dim // 10 + int(latent_dim % 10!=0), 10, figsize=(20, 14))
axes = axes.flatten()

from tqdm import tqdm
for i in tqdm(range(latent_dim), desc="Plotting histograms"):
    sns.histplot(flattened_motion[:, i].numpy(), ax=axes[i], kde=True)
    axes[i].set_title(f'Feature {i}')
    axes[i].set_xlabel('')
    axes[i].set_ylabel('')

plt.tight_layout()
plt.show()

# Plot box plots for each feature
plt.figure(figsize=(20, 6))
sns.boxplot(data=flattened_motion.numpy())
plt.title('Box Plot of Motion Features')
plt.xlabel('Feature Index')
plt.ylabel('Value')
plt.show()

In [ ]:
y=flattened_motion[:, 24]

In [ ]:
import numpy as np

def count_distribution(y, bins):
    pos_bins = np.array(bins)
    neg_bins = -pos_bins[::-1]
    all_bins = np.concatenate([neg_bins, [0], pos_bins])
    
    counts, _ = np.histogram(y, bins=all_bins)
    
    bin_labels = [f"-{b}" for b in bins[::-1]] + ["0"] + [f"+{b}" for b in bins]
    
    for label, count in zip(bin_labels, counts):
        print(f"{label}: {count}")

    return counts, bin_labels

# Define the bins
bins = [0.1, 0.2, 0.5, 1, 2, 5, 10, 20]

# Get the data for feature 24
y = flattened_motion[:, 24].numpy()

# Count the distribution
counts, labels = count_distribution(y, bins)

# Optionally, you can plot the distribution
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.bar(labels, counts)
plt.title('Distribution of Feature 24')
plt.xlabel('Bins')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Plot feature 24
plt.figure(figsize=(12, 6))
sns.histplot(flattened_motion[:, 24].numpy(), kde=True)
plt.title('Distribution of Feature 24')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()

# Box plot for feature 24
plt.figure(figsize=(8, 6))
sns.boxplot(y=flattened_motion[:, 24].numpy())
plt.title('Box Plot of Feature 24')
plt.ylabel('Value')
plt.show()

# Print summary statistics for feature 24
feature_24_summary = summary.iloc[24]
print("Summary statistics for Feature 24:")
print(feature_24_summary)


In [ ]:
def normalize_features(features):
    mean = features.mean(dim=0, keepdim=True)
    std = features.std(dim=0, keepdim=True)
    return (features - mean) / (std + 1e-8)  # Add small epsilon to avoid division by zero

normalized_features = normalize_features(all_coefs['shape'])

In [ ]:
import pandas as pd
import numpy as np
import torch

# Set display options to show all rows
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Calculate statistics
mean = flattened_motion.mean(dim=0)
std = flattened_motion.std(dim=0)
min_vals = flattened_motion.min(dim=0).values
max_vals = flattened_motion.max(dim=0).values
median = flattened_motion.median(dim=0).values  # Use .values to get the tensor
percentile_25 = torch.quantile(flattened_motion, 0.25, dim=0)
percentile_75 = torch.quantile(flattened_motion, 0.75, dim=0)

# Create the summary DataFrame
summary = pd.DataFrame({
    'Mean': mean.numpy(),
    'Std': std.numpy(),
    'Min': min_vals.numpy(),
    '25%': percentile_25.numpy(),
    'Median': median.numpy(),
    '75%': percentile_75.numpy(),
    'Max': max_vals.numpy()
})

# Print the full summary
print(summary)

# Reset display options to default (optional)
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

In [ ]:
import lmdb
import numpy as np
import os
import pickle
import torch
from tqdm import tqdm

# Specify the directory containing the LMDB file
lmdb_dir = '/mnt/e/data/diffposetalk_data/HDTF_TFHP-lmdb/'

# Check if the directory exists and contains the necessary files
if not os.path.isdir(lmdb_dir):
    print(f"The directory {lmdb_dir} does not exist.")
elif not os.path.isfile(os.path.join(lmdb_dir, 'data.mdb')):
    print(f"The data file 'data.mdb' is missing from {lmdb_dir}")
elif not os.path.isfile(os.path.join(lmdb_dir, 'lock.mdb')):
    print(f"The lock file 'lock.mdb' is missing from {lmdb_dir}")
else:
    print(f"Found LMDB environment in: {lmdb_dir}")

    # Try to open the LMDB environment
    # try:
    env = lmdb.open(lmdb_dir, readonly=True, lock=False)
    
    # Start a new read transaction
    with env.begin(write=False) as txn:
        # Get the number of keys in the database
        num_keys = txn.stat()['entries']
        print(f"Number of entries in the database: {num_keys}")

        # Read all entries
        all_data = []
        cursor = txn.cursor()
        for key, value in tqdm(cursor, total=num_keys, desc="Reading LMDB"):
            if key == b'metadata':
                metadata = pickle.loads(value)
                print("Metadata:", metadata)
            else:
                entry = pickle.loads(value)
                all_data.append(entry)



In [ ]:
key_count = {}
for entry in all_data:
    keys = tuple(entry.keys())  # Convert keys to tuple so it can be used as a dictionary key
    if keys in key_count:
        key_count[keys] += 1
    else:
        key_count[keys] = 1

# Print the results
for keys, count in key_count.items():
    print(f"Keys: {keys}, Count: {count}")

In [ ]:
# Process the data
coef_keys = ['shape', 'exp', 'pose']
all_coefs = {k: [] for k in coef_keys}
all_audio = []

for entry in all_data:
    if 'coef' in entry.keys():
        for k in coef_keys:
            all_coefs[k].append(entry['coef'][k])
    # Uncomment the following line if you want to process audio data
    # all_audio.append(torch.from_numpy(np.frombuffer(entry['audio'], dtype=np.float32)))

# Convert to tensors and stack arrays with shape[0] == 100
for k in coef_keys:
    valid_coefs = [coef for coef in all_coefs[k] if coef.shape[0] == 100]
    if valid_coefs:
        all_coefs[k] = np.stack(valid_coefs, axis=0)
        all_coefs[k] = torch.from_numpy(all_coefs[k])
    else:
        print(f"Warning: No valid coefficients found for {k}")
        all_coefs[k] = None

# Print shapes
print("\nData shapes:")
for k in coef_keys:
    if all_coefs[k] is not None:
        print(f"{k} shape: {all_coefs[k].shape}")
    else:
        print(f"{k}: No valid data")

# Calculate and print statistics
print("\nCoefficient statistics:")
for k in coef_keys:
    if all_coefs[k] is not None:
        print(f"\n{k}:")
        print(f"  Mean: {all_coefs[k].mean().item():.4f}")
        print(f"  Std: {all_coefs[k].std().item():.4f}")
        print(f"  Min: {all_coefs[k].min().item():.4f}")
        print(f"  Max: {all_coefs[k].max().item():.4f}")
    else:
        print(f"\n{k}: No valid data for statistics")

# Uncomment the following lines if you want to process audio data
# all_audio = torch.cat(all_audio, dim=0)
# print("\nAudio shape:", all_audio.shape)
# print("\nAudio statistics:")
# print(f"  Mean: {all_audio.mean().item():.4f}")
# print(f"  Std: {all_audio.std().item():.4f}")
# print(f"  Min: {all_audio.min().item():.4f}")
# print(f"  Max: {all_audio.max().item():.4f}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Set up the plots for all three coefficients
for coef_key in ['exp', 'shape', 'pose']:
    if all_coefs[coef_key] is not None:
        coef_data = all_coefs[coef_key]
        num_features = coef_data.shape[-1]
        
        # Calculate grid dimensions
        grid_rows = int(np.ceil(np.sqrt(num_features)))
        grid_cols = int(np.ceil(num_features / grid_rows))
        
        plt.figure(figsize=(20, 15))
        
        for feat_idx in range(num_features):
            # Extract the current feature dimension
            feature = coef_data[..., feat_idx].flatten()
            
            # Create a subplot for each feature
            plt.subplot(grid_rows, grid_cols, feat_idx + 1)
            
            # Plot the histogram
            plt.hist(feature.numpy(), bins=50, edgecolor='black')
            
            # Set title and labels
            plt.title(f'{coef_key.capitalize()} Feature {feat_idx + 1}')
            plt.xlabel('Value')
            plt.ylabel('Frequency')
            
            # Add some statistics to the plot
            plt.text(0.05, 0.95, f'Mean: {feature.mean():.4f}\nStd: {feature.std():.4f}', 
                     transform=plt.gca().transAxes, verticalalignment='top', fontsize=8)

        # Adjust the layout and display the plot
        plt.suptitle(f'Histogram of {coef_key.capitalize()} Coefficient Features', fontsize=16)
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.show()
    else:
        print(f"No valid data for '{coef_key}' coefficient")
